# RAG Evaluation

In [2]:
from operations.storages import ChromaDBOperations
import pandas as pd

chroma_oper = ChromaDBOperations()
documents = chroma_oper.get_all_documents()

df = pd.DataFrame([doc.get("document") for doc in documents], columns=["text"])

/home/wiktor/projects/rag-bot/src/operations/storages/chromadb_operations.py:34: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  self.embedder = HuggingFaceEmbeddings(model_name=embedder_dir)
/home/wiktor/projects/rag-bot/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
import giskard
from giskard.llm.client.openai import OpenAIClient
from agent.rag_components.llm import evaluation_llm

ai_client = OpenAIClient(model="gpt-4o-mini", client=evaluation_llm)

giskard.llm.set_default_client(ai_client)


/tmp/ipykernel_25166/709227187.py:7: DeprecationWarning: set_default_client is deprecated: https://docs.giskard.ai/en/latest/open_source/setting_up/index.html
  giskard.llm.set_default_client(ai_client)


In [3]:
from giskard.rag import KnowledgeBase
# from agent.rag_components.llm import openai_llm as llm

knowledge_base = KnowledgeBase(data=df[:8]) #, llm_client=llm)

2025-10-21 14:27:29,109 pid:25166 MainThread giskard.llm.embeddings INFO     No embedding model set though giskard.llm.set_embedding_model. Defaulting to openai/text-embedding-3-small since OPENAI_API_KEY is set.


In [14]:
from giskard.rag import generate_testset

testset = generate_testset(knowledge_base, 
                           num_questions=2)

INFO:giskard.rag:Finding topics in the knowledge base.
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
13:51:01 - LiteLLM:INFO: utils.py:1177 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler
13:51:01 - LiteLLM:INFO: cost_calculator.py:636 - selected model name for cost calculation: openai/text-embedding-3-small
INFO:LiteLLM:selected model name for cost calculation: openai/text-embedding-3-small
/home/wiktor/projects/rag-bot/.venv/lib/python3.10/site-packages/umap/umap_.py:2462: UserWarning: n_neighbors is larger than the dataset size; truncating to X.shape[0] - 1
  warn(
INFO:giskard.rag:Found 1 topics in the knowledge base.
Generating questions:   0%|          | 0/2 [00:00<?, ?it/s]13:51:01 - LiteLLM:INFO: utils.py:3085 - 
LiteLLM completion() model= gpt-4o; provider = openai
INFO:LiteLLM:
LiteLLM completion() model= gpt-4o; provider = openai
INFO:httpx:HTTP Request: POST https://api.o

In [6]:


test_df = testset.to_pandas()


for index, row in enumerate(test_df.head(3).iterrows()):
    print(f"Question {index + 1}: {row[1]['question']}")
    print(f"Reference answer: {row[1]['reference_answer']}")
    print("Reference context:")
    print(row[1]['reference_context'])
    print("******************", end="\n\n")

Question 1: What inspired the game mentioned in the context?
Reference answer: The game was inspired by Lizzie's political hero, the economist Henry George, whose ideas about putting the burden of taxation on wealthy landowners influenced its creation.
Reference context:
Document 1: In one corner were the Poor House and the Public Park, and across the board was the Jail. Another corner contained an image of the globe and an homage to Lizzie's political hero, the economist Henry George, whose ideas about putting the burden of taxation on wealthy landowners inspired the game: 'Labor upon Mother Earth Produces Wages.' Also included on the board were three words that have endured for more than a century after Lizzie scrawled them there: GO TO JAIL.Lizzie drew nine rectangular spaces along the edges of the board between each set of corners. In the centre of each nine-space grouping was a railroad, with spaces for rent or sale on either side.
******************

Question 2: In what way was L

In [16]:
testset.save("evaluation/test-set.jsonl")

In [4]:
from langchain.prompts import PromptTemplate

template = """
Respond to the question using only the information provided in the context. 
If the answer cannot be found, reply with “I don't know.”

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)


In [5]:
retriever = chroma_oper.get_retriever(k=4)

/home/wiktor/projects/rag-bot/src/operations/storages/chromadb_operations.py:46: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  return Chroma(


In [6]:
from langchain_core.output_parsers import StrOutputParser
from agent.rag_components import llm
from operator import itemgetter

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question")
    }
    | prompt
    | llm
    | StrOutputParser()
)

In [27]:
chain.invoke({"question": "What is a main point of the game?"})

INFO:httpx:HTTP Request: POST http://localhost:8800/api/v2/tenants/default_tenant/databases/default_database/collections/fc81f5a4-2787-4d44-a66b-2853a58b8eab/query "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


'A main point of the game is to compete against other players, with the goal of creating monopolies and crushing opponents.'

In [7]:
def answer_query_function(question):
    return chain.invoke({"question": question})

In [12]:
from giskard.rag import evaluate, RAGReport

report = RAGReport.load("evaluation/")

# report = evaluate(answer_query_function, testset=testset, knowledge_base=knowledge_base, llm_client=llm)

In [21]:
report.correctness_by_question_type()

,correctness
question_type,
complex,1.0
simple,1.0


In [13]:
display(report)

Loading BokehJS ...

/home/wiktor/projects/rag-bot/.venv/lib/python3.10/site-packages/umap/umap_.py:2462: UserWarning: n_neighbors is larger than the dataset size; truncating to X.shape[0] - 1
  warn(


# All metrics

In [3]:
from operations.storages import ChromaDBOperations
import pandas as pd

chroma_oper = ChromaDBOperations()
documents = chroma_oper.get_all_documents()

df = pd.DataFrame([doc.get("document") for doc in documents], columns=["text"])


from giskard.rag import KnowledgeBase
from agent.rag_components import llm

knowledge_base = KnowledgeBase(data=df, llm_client=llm)


from langchain.prompts import PromptTemplate

template = """
Respond to the question using only the information provided in the context. 
If the answer cannot be found, reply with “I don't know.”

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)



retriever = chroma_oper.get_retriever(k=4)



from langchain_core.output_parsers import StrOutputParser
from operator import itemgetter

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question")
    }
    | prompt
    | llm
    | StrOutputParser()
)

def answer_query_function(question):
    return chain.invoke({"question": question})



/home/wiktor/projects/rag-bot/src/operations/storages/chromadb_operations.py:34: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  self.embedder = HuggingFaceEmbeddings(model_name=embedder_dir)


2025-10-21 13:30:09,839 pid:12424 MainThread giskard.llm.embeddings INFO     No embedding model set though giskard.llm.set_embedding_model. Defaulting to openai/text-embedding-3-small since OPENAI_API_KEY is set.


/home/wiktor/projects/rag-bot/src/operations/storages/chromadb_operations.py:46: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  return Chroma(


In [8]:
from giskard.rag import QATestset

testset = QATestset.load("evaluation/test-set.jsonl")

In [9]:
print(testset.samples)

[QuestionSample(id='c56d7b66-648b-4db0-9325-8a55af780395', question="What was Lizzie's vision characterized by?", reference_answer='Her vision was an embrace of dualism and contained a contradiction within itself, a tension trying to be resolved between opposing philosophies.', reference_context="Document 4: Her vision was an embrace of dualism and contained a contradiction within itself, a tension trying to be resolved between opposing philosophies. However, and of course unbeknownst to Lizzie at the time, it was the monopolist rules that would later capture the public's imagination.After years of tinkering, writing and pondering her new creation, Lizzie entered the US Patent Office on 23 March 1903 to secure her legal claim to the Landlord's Game.", conversation_history=[], metadata={'question_type': 'simple', 'seed_document_id': 4, 'topic': 'Others'}, agent_answer=None, correctness=None), QuestionSample(id='fb0e12ff-90f4-4b78-b4bc-45df72344efb', question="Could you detail the two di

In [10]:
from giskard.rag.metrics.ragas_metrics import (
    ragas_context_precision,
    # ragas_faithfulness,
    # ragas_answer_relevancy,
    ragas_context_recall,
)
# from giskard.rag.metrics import correctness_metric

from giskard.rag import evaluate


report = evaluate(
    answer_query_function, 
    testset=testset, 
    knowledge_base=knowledge_base, 
    metrics=[
        # correctness_metric,
        ragas_context_recall,
        ragas_context_precision,
        # ragas_faithfulness,
        # ragas_answer_relevancy
        ]
    )

CorrectnessMetric evaluation:   0%|          | 0/2 [00:00<?, ?it/s]


LLMGenerationError: Error while evaluating the agent

In [14]:
display(report)

NameError: name 'report' is not defined